In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from filtering import between
from datetime import date, datetime

folder = Path("base")

In [3]:
from preprocess_utils import load_jsonb, save_jsonb 
from tqdm import tqdm
from urllib.parse import urlparse
for art in load_jsonb("final.jsonb"):
    url = art["url"]
    parsed = urlparse(url).netloc.split(".")
    server = parsed[-1]
    if server == "cz":
        server = parsed[-2]

    save_jsonb([art], folder / f"{server}.jsonb", "a", show_progress=False)



    



FileNotFoundError: [Errno 2] No such file or directory: 'final.jsonb'

In [6]:
# Add from cz:403 to aktualne becauose of the same domain

for art in load_jsonb(folder / "cz:443.jsonb"):
    save_jsonb([art], folder / "aktualne.jsonb", "a", show_progress=False)


100%|██████████| 1/1 [00:00<00:00, 1490.51it/s]


In [16]:
from preprocess_utils import articles_num
print(list(articles_num(folder)))

[('denik.jsonb', 1636639), ('seznamzpravy.jsonb', 77442), ('irozhlas.jsonb', 201681), ('idnes.jsonb', 530855), ('aktualne.jsonb', 165799), ('novinky.jsonb', 549661)]


In [12]:
# Custom config for each server
configs = {
    "idnes": {},
    "denik": {},
    "aktualne": {},
    "irozhlas": {},
    "seznamzpravy": {},
    "novinky": {},
}

default = {
    "article_length": between(300, None),
    "avg_word_length": between(4.0, None),
    "num_words_ratio": between(0.11, 0.22),
    "headline_length": between(20, None),
    "brief_length": between(40, None),
    "non_alpha_ratio": between(None, 0.045),
    "date": between(date(2000, 1, 1), date(2022, 8, 31)),
}

In [14]:
from typing import Callable, List
from filtering import create_filter_by_stats, create_filter_by_cz_lang, create_config 
from preprocess_utils import load_jsonb, save_jsonb
from postprocessing_utils import postprocess_authors, filter_author, add_server, postprocess_brief, postprocess_category, postprocess_headline, postprocess_date, as_Article, JSONArticleEncoder

from functools import reduce
filtered_folder = Path("article_json_filtered")
servers = [f for f in folder.iterdir()]

ratio = 0.75
for f in servers:
    filters = [
        create_filter_by_cz_lang(),
        create_filter_by_stats(create_config(configs[f.stem], default))
    ]

    augmentations: List[Callable[[dict], dict]] = [add_server(f.stem), filter_author, postprocess_authors, postprocess_brief, postprocess_category, postprocess_headline, postprocess_date]


    articles = load_jsonb(f)
    articles = filter(lambda x: all(f(x) for f in filters), articles)


    # Apply as df filters as cold be faster    
    articles = (reduce(lambda x, f: f(x), augmentations, article) for article in articles)

    as_articles = (as_Article(article) for article in articles)
    save_jsonb(as_articles, filtered_folder / f"{f.stem}.jsonb", encoder=JSONArticleEncoder, show_progress=False)

         



    

100%|██████████| 549661/549661 [31:07<00:00, 294.28it/s] 


In [15]:

from preprocess_utils import articles_num
print(list(articles_num(filtered_folder)))

[('denik.jsonb', 1185153), ('seznamzpravy.jsonb', 67379), ('irozhlas.jsonb', 172458), ('idnes.jsonb', 314479), ('aktualne.jsonb', 118210), ('novinky.jsonb', 372545)]


In [18]:
# category fix
fixed_folder = Path("category_fixed")
for f in filtered_folder.iterdir():
    articles = load_jsonb(f)
    articles = (postprocess_category(article) for article in articles)
    save_jsonb(articles, fixed_folder / f.name, encoder=JSONArticleEncoder, show_progress=True)


278it [00:32, 12.33it/s]

In [17]:
# Create splits 70/20/10 by date
from datetime import date
import random
output_folder = Path("final_dataset")
servers = [f for f in fixed_folder.iterdir()]

all_articles = (article for f in servers for article in load_jsonb(f))
none_articles = (article for article in all_articles if article["date"] is None)
articles = (article for article in all_articles if article["date"] is not None)
sorted_articles = sorted(articles, key=lambda x: date.fromisoformat(x["date"]))
# Randomly insert none articles
for none_art in none_articles:
    index = random.randint(0, len(sorted_articles))
    sorted_articles.insert(index, none_art)

train = sorted_articles[:int(len(sorted_articles) * 0.7)]
dev = sorted_articles[int(len(sorted_articles) * 0.7):int(len(sorted_articles) * 0.9)]
test = sorted_articles[int(len(sorted_articles) * 0.9):]

save_jsonb(train, output_folder / "train.jsonb", show_progress=True)
save_jsonb(dev, output_folder / "dev.jsonb", show_progress=True)
save_jsonb(test, output_folder / "test.jsonb", show_progress=True)



 22%|██▏       | 259652/1185153 [00:55<03:19, 4639.94it/s]


KeyboardInterrupt: 